In [20]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error
from pathlib import Path
import os
import pickle
from pathlib import Path
import os
from tensorflow import keras
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

plt.rcParams["figure.figsize"] = (30, 10)
plt.rcParams["figure.dpi"] = 600

In [10]:
TICKER = "SMP"
MODEL_VERSION = 0.1
MODEL_NAME = "lstm"

In [11]:
load_path = Path.cwd().parents[0] / "data"
models_path = Path.cwd() / "data" / "lstm" / "versions"

with open(load_path / f"data_{TICKER}.pickle", "rb") as handle:
    data = pickle.load(handle)

with open(load_path / f"scalers_{TICKER}.pickle", "rb") as handle:
    scalers = pickle.load(handle)

In [12]:
data.keys()

dict_keys(['X_list_train', 'X_list_test', 'Y_preds_real_list_train', 'Y_preds_real_list_test', 'Y_whole_real_list_train', 'Y_whole_real_list_test'])

In [13]:
model = keras.models.load_model(models_path / f"{MODEL_NAME}_{MODEL_VERSION}")
preds = model.predict(data["X_list_test"])
backtest_path = Path(os.path.abspath("")).parents[1] / "backtesting"
actual_values = data["Y_preds_real_list_test"]
test_preds = scalers["y_scaler"].inverse_transform(preds)
test_true = scalers["y_scaler"].inverse_transform(actual_values)

1/1 [==============================] - 1s 576ms/step


In [14]:
mean_absolute_error(y_true=test_true, y_pred=test_preds)


889.4000748697916

In [15]:
mean_absolute_percentage_error(y_true=test_true, y_pred=test_preds)

0.20834272915843566

In [22]:
mean_squared_error(y_true=test_true, y_pred=test_preds) ** 0.5

927.1541246724522

In [25]:
import numpy as np
smape = np.mean(2 * np.abs(test_preds - test_true) / (np.abs(test_true) + np.abs(test_preds))) * 100
smape

23.424436236075866